# Estimating the prevalence of CoViD19 in Italy

During the pandemic I wrote with my friends Fabio Verachi and Luciano Lanzi a paper that proposed a quantitative modeling approach to estimate the direct costs of CoViD19 in Italy. It was based on a standard epidemic model (a variant of SIR) and a multi state model (to calculate hospital and ICU occupation rates).
You can find the preprint on medRxiv, [here](https://www.medrxiv.org/content/10.1101/2020.05.28.20115790v1); the paper has been accepted by the Risk Management Magazine, you can find the published version [here](https://www.aifirm.it/wp-content/uploads/2020/08/RMM_2020_2.pdf). The model was fitted on data available on May 4th 2020.

Recently ISTAT published the [results](https://www.istat.it/it/files//2020/08/ReportPrimiRisultatiIndagineSiero.pdf) of a serological test campaign in Italy that estimated prevalence in the time span from May 25 to July 15 at 2.5% of population (confidence interval from 2.3% to 2.6%), 1.482.377 people.

This results are quite a surprise. During the pandemic statistic institutions in Italy (like ISPI) estimated a much larger prevalence (4.4%, 2.642.326) and their results had great diffusion (see fo example the Financial Times,https://www.ft.com/content/7c312d8a-fcfe-4ce7-94d5-f681221e6042).

I will show here that the numbers implied in our model are in line with ISTAT observations

Let's start defining our model (just had fun trying to not to use numpy):

In [1]:
class SIR_Model:
    def __init__(self,beta,D1,D2,PCatch,LockDnT,LockDnBeta):
        self.beta=beta
        self.D1=D1
        self.D2=D2
        self.PCatch=PCatch
        self.LockDnT=LockDnT
        self.LockDnBeta=LockDnBeta 
    def Diff_SIR(self,Status,isBefore=False):
        CurrentBeta=self.LockDnBeta
        if (isBefore):
            CurrentBeta=self.beta
        PFree=1-self.PCatch
        Gamma1=1/self.D1
        Gamma2=1/self.D2
        # Status=[S,I,R1,R2]
        # Output=[dS,dI,dR1,dR2]
        return [-Status[0]*Status[1]*CurrentBeta,
                Status[0]*Status[1]*CurrentBeta-(self.PCatch*Gamma1+PFree*Gamma2)*Status[1],
                self.PCatch*Gamma1*Status[1],
                PFree*Gamma2*Status[1]]   
    def NextStep(self,Status,isBefore=False):
        # Runge-Kutta integration
        D1=self.Diff_SIR(Status,isBefore)
        S1=[0.5*x+y for x,y in zip(D1,Status)]
        D2=self.Diff_SIR(S1,isBefore)
        S2=[0.5*x+y for x,y in zip(D2,Status)]
        D3=self.Diff_SIR(S2,isBefore)
        S3=[x+y for x,y in zip(D2,Status)]
        D4=self.Diff_SIR(S3,isBefore)
        increment=[w/6+x/3+y/3+z/6 for w,x,y,z in zip(D1,D2,D3,D4)]
        NewStatus=[x+y for x,y in zip(Status,increment)]
        return NewStatus
    def SIR_Simulation(self,Status,StartDays=0,NumOfDays=180):
        Simulation=[[0,Status]]
        for i in range(NumOfDays):
            if i<=self.LockDnT:
                NewStatus=self.NextStep(Status,True)
            else:
                NewStatus=self.NextStep(Status)
            Simulation.append([i,NewStatus])
            Status=NewStatus
        return(Simulation)

For details on the model see the paper: basically we simulate the population among with the pandemic spreads S (that is less than the total population, since we had a lockdown), the infected I, and the Removed of type one (infection detected) or type 2 (infection silent): the idea is that the type 2 infected are able to spread infection.

In order to simulate death toll we need to implement also the multi state model. Again, see the paper: type one removed can either heal or die after some time, during with they evolve trhough different states (alive at home, alive at hospital, alive in ICU, dead, alive and healed). In this notebook we are interested only in the ones that die. Notice that in our model no type 2 infected die: the idea is that you don't die of CoViD in hours, so anybody that is going to dye is detected.

In [2]:
class MultiStateModel:
    def __init__(self,DeathProbs,HealProbs,Time1,Time2):
        # Destiny of a covid infected simulated human
        # that is catched by Health Guard system is
        # divided in three periods:
        # Catch -> Catch + Time1
        # Dies with prob DeathProbs[0]
        # Catch + Time1 -> Catch + Time2
        # Dies with prob DeathProbs[1]
        # Heals with prob HealProbs[0]
        # Catch + Time2 -> infinity
        # Dies with prob DeathProbs[2]
        # Heals with prob HealProbs[1]
        # All probabilities are "per day"
        self.DP1=DeathProbs[0]
        self.DP2=DeathProbs[1]
        self.DP3=DeathProbs[2]
        self.HP2=HealProbs[0]
        self.HP3=HealProbs[1]
        self.Time1=Time1
        self.Time2=Time2
    def DeathSimulation(self,CatchedFlow,StartLoad,Memory=200):
        # Set Initial status
        Status=[]
        CumDeaths=[0]
        for i in range(Memory):
            if i<len(StartLoad):
                Status.append(StartLoad[i])
            else:
                Status.append(0)
        # Evolution rules
        # Calculate outflows
        for t in range(len(CatchedFlow)):
            # Calculate outflows
            Pop1=0
            Pop2=0
            Pop3=0
            for i in range(Memory):
                if i<self.Time1:
                    Pop1+=Status[i]
                if i<self.Time2:
                    Pop2+=Status[i]
                Pop3+=Status[i]
            Pop3=Pop3-Pop2-Pop1
            Pop2=Pop2-Pop1
            Outflow1=np.int(self.DP1*Pop1)
            Outflow2=np.int(self.DP2*Pop2+self.HP2*Pop2)
            Outflow3=np.int(self.DP3*Pop3+self.HP3*Pop3)
            Deaths=np.int(self.DP1*Pop1+self.DP2*Pop2+self.DP3*Pop3)
            CumDeaths.append(CumDeaths[-1]+Deaths)
            # GrowOneDay and inflow
            for i in range(Memory):
                j=Memory-i-1
                if j>0:
                    Status[j]=Status[j-1]
                    Status[0]=CatchedFlow[t]
            # Outflow group 3
            for i in range(Memory):
                j=Memory-i-1
                if Status[j]>0 and Outflow3>0:
                    if Status[j]<Outflow3:
                        Outflow3=Outflow3-Status[j]
                        Status[j]=0
                    else:
                        Status[j]=Status[j]-Outflow3
                        Outflow3=0
            # Outflow group 2
            for i in range(Memory):
                j=Memory-i-1
                if Status[j]>0 and Outflow2>0 and j<self.Time2:
                    if Status[j]<Outflow2:
                        Outflow2=Outflow2-Status[j]
                        Status[j]=0
                    else:
                        Status[j]=Status[j]-Outflow2
                        Outflow2=0
            # Outflow group 1
            for i in range(Memory):
                j=Memory-i-1
                if Status[j]>0 and Outflow1>0 and j<self.Time1:
                    if Status[j]<Outflow1:
                        Outflow1=Outflow1-Status[j]
                        Status[j]=0
                    else:
                        Status[j]=Status[j]-Outflow1
                        Outflow1=0
        return CumDeaths

Now we have all definitions, so let's start playing. Using the parameters of the paper, we instatiate models:

In [3]:
SIRModel=SIR_Model(0.27704,5.2,15,0.05375,13+38,0.16632)
StateModel=MultiStateModel([0.03557,0.001298,0.001566],[0.032750,0.015831],4,9)

Let's start the epidemiological simulation:

In [4]:
Initial_Status=[1-5e-6,5e-6,0,0]
SIRDynamics=SIRModel.SIR_Simulation(Initial_Status)

Since in the paper we estimated a Dimension of the pandemic (the total people that are around the infected) of 1.995.898, and that May 25 and July 15 are the 129th and 180th simulated days, the total removed and infected people at start and end of the ISTAT campaing are:

In [5]:
import numpy as np
Dimension=1995898
TotalCount=[0,0]
TotalCount[1]=np.int((SIRDynamics[180][1][1]+SIRDynamics[180][1][2]+SIRDynamics[180][1][3])*Dimension)
TotalCount[0]=np.int((SIRDynamics[129][1][1]+SIRDynamics[129][1][2]+SIRDynamics[129][1][3])*Dimension)

In [6]:
TotalCount[0]

1723368

In [7]:
TotalCount[1]

1742957

Since we want alive infected, we need to estimate deaths. The state model needs the infected detection inflow. It is annoying since the SIR simulation starts before the infection was found in italy, so there's an offset. Let's start putting in the time series of inflows observed from 2020-02-25 (vs 24) to 2020-05-04 (vs 03) (data coming from Protezione Civile GitHub [repository](https://github.com/pcm-dpc/COVID-19)):

In [8]:
TrueInflows=[93,78,250,238,240,566,342,466,587,769,778,1247,1492,1797,977,
             2313,2651,2547,3497,3590,3233,3526,4207,5322,5986,6557,5560,4789,
             5249,5210,6153,5959,5974,5217,4050,4053,4782,4668,4585,4805,4316,
             3599,3039,3836,4204,3951,4694,4092,3153,2972,2667,3786,3493,3491,
             3047,2256,2729,3370,2646,3021,2357,2324,1739,2091,2086,1872,1965,
             1900,1389,1221]

We need to combine this with the simulated data for inflows after May 4th:

In [9]:
SimulatedInFlow=[]
for i in range(len(SIRDynamics)):
    if i>0:
        CatchedIncrement=np.int(Dimension*(SIRDynamics[i][1][2]-SIRDynamics[i-1][1][2]))
        SimulatedInFlow.append(CatchedIncrement)
JoinedInFlow=TrueInflows
for i in range(51):
    JoinedInFlow.append(SimulatedInFlow[108+i])

Now we can feed the joined inflow estimation into the state model and find the expected deaths:

In [10]:
StateModel=MultiStateModel([0.03557,0.001298,0.001566],[0.032750,0.015831],4,9)
DeceaseSeries=StateModel.DeathSimulation(JoinedInFlow,[46,46,46,46,45])
DeadCount=[0,0]
DeadCount[1]=DeceaseSeries[121]
DeadCount[0]=DeceaseSeries[70]

The expected alive infected at the starting of ISTAT campaign are:

In [11]:
TotalCount[0]-DeadCount[0]

1689990

While at the end are:

In [12]:
TotalCount[1]-DeadCount[1]

1699008

In terms of prevalence, our central value is:

In [13]:
(TotalCount[0]+TotalCount[1]-DeadCount[0]-DeadCount[1])/2/60030201

0.028227441717211643

We have a 0.32% error respect to central estimate of ISTAT! I am proud to conclude that our model is quite accurate!